# First protocol
_Code written and runs in python 3.11.0. Modify environment variables and queries as needed._  
_Please use venv_

## Protocol
· Start with keywords:

- Smartchain

- Nft

- Airdrop

- Crypto

- …etc.

1. Sample up to 10k tweets containing at least one term from 100 random hours from the past year (so 1M tweets)

2. Determine the most engaged (top) with users from this combined sample (100 or 1000)

3. Pull up to 1000 comments for each top user

4. Determine top users whose comments mention at least three users other than the top user

5. Expand top user sample if we don’t have at least 100 airdrop seeders

6. Time series chart plots:

7. Top user activity

8. Airdrop seeder activity

9. Negative reaction activity? (based on sentiment analysis of replies to airdrop messages)

10. External crypto value signals (from where?)



# Dependencies
Run the following commands in the terminal to install the required packages

$pip install requests  
  
$pip install pandas  
  
$pip install datetime  
  
$pip install python-dateutil
  

--------------------  

Crypto packages/api's:
https://medium.com/codex/10-best-resources-to-fetch-cryptocurrency-data-in-python-8400cf0d0136

https://www.alphavantage.co/documentation/


Financial tools/packages:
https://twitter.com/pyquantnews/status/1568029967052640256?t=EthvrNWmYhAFDVOhRoDxrQ&s=03

https://pmorissette.github.io/ffn/quick.html#data-retrieval


# Authentication step
In the code cell below replace bearer_token with your bearer token. Run the cell, then delete your bearer token.
This creates the token as an environment variable to be used under the name TOKEN. The token can then be removed so that others do not have access to your token when code is shared via GitHub. I will change this to dotenv and a .gitignore file later I just havent done that yet.

In [24]:
os.environ['TOKEN'] = ''

In [25]:
"""This cell creates functions to be used for authentication as well as creating endpoints."""
import requests

def auth():
    """Retrieves your bearer token."""
    return os.getenv('TOKEN')

def create_headers(bearer_token):
    """Creates headers for proper authentication from an API request."""
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def create_full_search_url(keyword: str, start_date: list[str], end_date: list[str], max_results: int = 100):
    """Creates queries and params for a full archive search url."""
    search_url: str = "https://api.twitter.com/2/tweets/search/all" 

    # change params to desired params
    query_params: dict = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'referenced_tweets.id.author_id',
                    'tweet.fields': 'id,author_id,conversation_id,created_at,in_reply_to_user_id,lang,public_metrics,referenced_tweets,source,text',
                    #'user.fields': 'id,name,public_metrics,username,verified',
                    #'place.fields': 'country',
                    'next_token': {}}
    return (search_url, query_params)

def search_tweet_author_id_url(id: str):
    """Creates queries to find the author id of a tweet's author."""
    search_url: str = f"https://api.twitter.com/2/tweets/{id}" 

    # change params to desired params
    query_params: dict = {'tweet.fields': 'author_id'}
    return (search_url, query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    """This takes a url from a url creation function and the params from the same function as well as an optional next token and returns a json object response from the endpoint."""
    params['next_token'] = next_token   # if a next token is found this will assign it to params 'next_token' key
    response = requests.request("GET", url, headers = headers, params = params)
    print("\nEndpoint Response Code: " + str(response.status_code)) # prints the enpoint response code for debugging help
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [26]:
import datetime
import random
"""This code cell contains two functions (is_leap_year and random_date) which help generate a random one hour date range when random_date() is called"""


def is_leap_year(year: int):
    """Returns True if the given year in typical four digit year format (i.e. 2023) is a leap year, False otherwise."""
    if year % 4 == 0:
        if year % 100 == 0:
            if year % 400 == 0:
                return True
            else:
                return False
        else:
            return True
    else:
        return False

def sort_timestamps(timestamps: list):
    """This function takes the list of start and end times in rfc 3339 format, returned from the return_n_random_hour_ranges_sorted function, and sorts the lists in chronological order."""
    # Convert timestamps to datetime objects
    datetimes = [datetime.datetime.fromisoformat(ts) for ts in timestamps]
    # Sort datetime objects
    datetimes.sort()
    # Convert sorted datetime objects back to timestamps
    sorted_timestamps = [dt.strftime("%Y-%m-%dT%H:%M:%SZ") for dt in datetimes]
    return sorted_timestamps

def random_date():
    """Generate a random one hour date range within the last year in RFC 3339 format to be used with twitter API."""
    month = random.randint(1, 12)
    year = random.randint(datetime.datetime.now().year - 1, datetime.datetime.now().year)
    if month <= datetime.datetime.now().month:
        year = datetime.datetime.now().year
    else:
        year = datetime.datetime.now().year - 1
    if month == datetime.datetime.now().month:
        if datetime.datetime.now().day <= 2:
            day = 1
        else:
            day = random.randint(1, datetime.datetime.now().day - 1)
    elif month == 2:
        if is_leap_year(year):
            day = random.randint(1, 29)
        else:
            day = random.randint(1, 28)
    elif month in [1, 3, 5, 7, 8, 10, 12]:
        day = random.randint(1, 31)
    else:
        day = random.randint(1, 30)
    hour = random.randint(0, 23)
    start_time = datetime.datetime(year, month, day, hour)
    end_time = start_time + datetime.timedelta(hours=1)
    start_timestamp = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_timestamp = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    return start_timestamp, end_timestamp

def return_n_random_hour_ranges_sorted(n: int) -> list:
    """This returns two SORTED lists of start times and end times (where each index of the end time is one hour after the same index in start time)."""
    """This function returns n number of one hour ranges."""
    start_time1_list: list = list()
    end_time1_list: list = list()
    for i in range(0, n):
        s1, s2 = random_date()
        while s1 in start_time1_list:
            s1, s2 = random_date()
        start_time1_list.append(s1)
        end_time1_list.append(s2)

    sorted_start = sort_timestamps(start_time1_list)
    sorted_end = sort_timestamps(end_time1_list)
    return (sorted_start, sorted_end)

In [39]:
import json
import pandas as pd
import csv
from typing import Optional
import time

def tweets_per_range(keyword: str, start_times_list: list, end_times_list: list, results_per_range: int, next_token: Optional[str] = None) -> None:
    """This function takes a keyword(s), a list of start times and end times, and a integer amount of results per range."""
    """It then creates json files for each time range and stores the tweet results in those json files."""
    
    # AUTHENTICATION
    bearer_token = auth()
    headers: dict[str, str] = create_headers(bearer_token)
    # AUTHENTICATION

    json_obj_by_time_range: dict[str, dict] = dict() # creates a dictionary to which the time range will be created as a key and can therefore be found while in the json file
    max_results: int = 500 # this is the max results per request the twitter API allows and should be left at 500
    total_tweets_for_func_call: int = 0

    # Loops through the time ranges in a list
    for i in range(0, len(start_times_list)):
        json_obj_by_time_range: dict[str, dict] = dict()
        print("top of FOR loop")
        total_count = 0 # Tracks
        
        # Creates url and connects to endpoint then assignts the json object API response to json_obj_response
        url = create_full_search_url(keyword, start_times_list[i], end_times_list[i], max_results)
        json_obj_response = connect_to_endpoint(url[0], headers, url[1], next_token) # prints response code
        print(f"Outer for loop enpoint called for list index {i} / {len(start_times_list) - 1}") # for quality control
        json_obj_response.pop('includes', None) # removes 'includes' key which is a negative externality of calling 'referenced_tweets.id.author_id' expansion
        json_obj_response['time'] = [start_times_list[i], end_times_list[i]] # adds a 'time' key to the json_obj_response so that the time range of all tweets can be found by calling json_object_response['time']

        # Appends the json object API response to the json_obj_by_time_range dictionary
        json_obj_by_time_range[f'time_range_{i}'] = json_obj_response
        total_count += json_obj_response['meta']['result_count'] # increases the total count counter by the results count in the first API 'GET'
        print(f"endpoint called and data collected: {total_count} / {results_per_range} tweets in this range scraped")
        time.sleep(5) # time.sleep implementations are seen throughout the code to avoid hitting rate limits of twitter API
        
        # If there were insifficient results from first 'GET' request to meet the results per range value then the API begins to paginate to scrape more resutls
        while total_count <= results_per_range: 
            # Checks for next token in json response
            print("top of WHILE loop")
            if 'next_token' in json_obj_response['meta']: 
                
                next_token: str = json_obj_response['meta']['next_token'] # assigns 'next_token' to next_token: str object for easy use

                # Creates url and connects to endpoint then assignts the JSON API response to json_obj_esponse
                json_obj_response = connect_to_endpoint(url[0], headers, url[1], next_token) # prints response code
                print(f"While loop enpoint called: index {i} / {len(start_times_list) - 1}") # for quality control
                
                next_token = None # ensures next token does not get passed into another function call 

                if 'data' in json_obj_response:
                    # Loops through dictionaries in json_obj_response and appends them to the main json file
                    # This is done because while theoretically the entire item count be appended at once, certain python vectorizing methods might cause disagreeable types. Looping through items avoids this happening
                    for item in json_obj_response['data']:
                        json_obj_by_time_range[f'time_range_{i}']['data'].append(item)
                    
                    total_count += json_obj_response['meta']['result_count'] # increments the result count to match the total results currently aquired
                    json_obj_by_time_range[f'time_range_{i}']['meta']['result_count'] = total_count # changes the result count 'key' to meet the result count of all data

                    if json_obj_by_time_range[f'time_range_{i}']['meta']['result_count'] > results_per_range:
                        del json_obj_by_time_range[f'time_range_{i}']['data'][results_per_range:]
                        json_obj_by_time_range[f'time_range_{i}']['meta']['result_count'] = len(json_obj_by_time_range[f'time_range_{i}']['data'])

                    print(f"data key found and data appended: {total_count} / {results_per_range} tweets in this range scraped") # quality control
                    print(f"list index {i} / {(len(start_times_list) - 1)}")

                else:
                    print("empty next token") # quality control
                    print(f"max results scraped: {total_count} / {results_per_range} tweets in this range scraped") # quality control
                    print(f"list index {i} / {(len(start_times_list) - 1)} scraping over. Total tweets will be less than desired")
                    break
            
            else:
                print("No more tweets to scrape, total tweets will be less than amount desired.") # quality control
                print(f"total results {total_count}") # quality control
                next_token = None # ensures next token does not get passed into another function call
                break # exits while loop and calls for the next time range or terminates entire process
            

            time.sleep(5) # for rate limit 
        total_tweets_for_func_call += len(json_obj_by_time_range[f'time_range_{i}']['data'])
        print(f"{total_tweets_for_func_call} tweets scraped in entire function call")
        time.sleep(5) # for rate limit

            
        json_to_file = json.dumps(json_obj_by_time_range) # converts results from above code to a serialized json obj
        # Creates json file in directory of program and writes serialized json obj to the newly made file
        with open(f"data_range_{i}.json", "w") as outfile:
            outfile.write(json_to_file)
        

"""This function takes all of the json files of tweet data and creates a sorted dictionary of the most appearing tweets."""
"""A post OR a retweet counts as ONE occurence of a tweet."""
def analyze_top_appearing_tweets_in_data(max_int_of_json: int) -> dict: # max_int_of_json is the max int of json files in your directory, this function will paginate through them
    tweet_metrics_dict: dict = dict()
    for i in range(0, max_int_of_json + 1):

        # opens json file and assigns serialized json data to data_file 
        f = open(f'data_range_{i}.json')
        data_file = json.load(f)

        for item in data_file[f'time_range_{i}']['data']: # loops through the tweets in json file and adds to a dict key of that tweet's id if the tweet appears
            # Increments each tweet id 'key' in tweet_metrics_dict by one for each appearance of a tweet or each retweet of that tweet
            if 'referenced_tweets' in item and item['referenced_tweets'][0]['type'] == "retweeted":
                original_tweet_id_from_retweet = item['referenced_tweets'][0]['id']
                if original_tweet_id_from_retweet in tweet_metrics_dict:
                    tweet_metrics_dict[original_tweet_id_from_retweet] += 1
                else:
                    tweet_metrics_dict[original_tweet_id_from_retweet] = 1
            else:
                this_tweet_id = item['id']
                if this_tweet_id in tweet_metrics_dict:
                    tweet_metrics_dict[this_tweet_id] += 1
                else:
                    tweet_metrics_dict[this_tweet_id] = 1

        f.close() # closes the json file

    sorted_dict = {} # creates a dictionary to assist with sorting tweet id's by the most appearances
    sorted_keys = sorted(tweet_metrics_dict, key=tweet_metrics_dict.get, reverse=True)  # sorts the keys by highes value (most tweet appearances)

    for w in sorted_keys:
        sorted_dict[w] = tweet_metrics_dict[w] # sorts the dictionary by value from greates to least
    return sorted_dict # returns sorted dict


def return_stats_of_file(num_of_file: int):
    f = open(f'data_range_{num_of_file}.json')
    data_file = json.load(f)
    print(f"There are {len(data_file[f'time_range_{num_of_file}']['data'])} tweets in this data file")
    print(data_file[f'time_range_{num_of_file}']['meta'])
    print(data_file[f'time_range_{num_of_file}']['time'])
    f.close()


def analyze_top_retweeted_tweets_in_data(max_int_of_json: int) -> dict: # max_int_of_json is the max int of json files in your directory, this function will paginate through them
    tweet_metrics_dict: dict = dict()
    for i in range(0, max_int_of_json + 1):

        # opens json file and assigns serialized json data to data_file 
        f = open(f'data_range_{i}.json')
        data_file = json.load(f)

        for item in data_file[f'time_range_{i}']['data']: # loops through the tweets in json file and adds retweet count to a dict key of that tweet's id 
            # Creates a key for each unique tweet id and assigns its retweet count to that key
            if 'referenced_tweets' in item and item['referenced_tweets'][0]['type'] == "retweeted":
                original_tweet_id_from_retweet = item['referenced_tweets'][0]['id']
                if original_tweet_id_from_retweet not in tweet_metrics_dict:
                    retweet_count = item['public_metrics']['retweet_count']
                    tweet_metrics_dict[original_tweet_id_from_retweet] = retweet_count
            else:
                this_tweet_id = item['id']
                if this_tweet_id not in tweet_metrics_dict:
                    tweet_metrics_dict[this_tweet_id] = item['public_metrics']['retweet_count']
        f.close() # closes the json file

    sorted_dict = {} # creates a dictionary to assist with sorting tweet id's by the most appearances
    sorted_keys = sorted(tweet_metrics_dict, key=tweet_metrics_dict.get, reverse=True)  # sorts the keys by highes value (most retweets)

    for w in sorted_keys:
        sorted_dict[w] = tweet_metrics_dict[w] # sorts the dictionary by value from greates to least
    return sorted_dict # returns sorted dict


# Main Working Cell

The below cell is intended to be the final cell which ties together all functions into the 'solution' to the protocol

In [35]:
start_list, end_list = return_n_random_hour_ranges_sorted(100)
json_final_data = tweets_per_range("Smartchain OR Airdrop OR Crypto OR Nft", start_list, end_list, 10000)

top of FOR loop

Endpoint Response Code: 200
Outer for loop enpoint called for list index 0 / 99
endpoint called and data collected: 332 / 10000 tweets in this range scraped
top of WHILE loop

Endpoint Response Code: 200
While loop enpoint called: index 0 / 99
data key found and data appended: 686 / 10000 tweets in this range scraped
list index 0 / 99
top of WHILE loop

Endpoint Response Code: 200
While loop enpoint called: index 0 / 99
data key found and data appended: 1006 / 10000 tweets in this range scraped
list index 0 / 99
top of WHILE loop

Endpoint Response Code: 200
While loop enpoint called: index 0 / 99
data key found and data appended: 1347 / 10000 tweets in this range scraped
list index 0 / 99
top of WHILE loop

Endpoint Response Code: 200
While loop enpoint called: index 0 / 99
data key found and data appended: 1693 / 10000 tweets in this range scraped
list index 0 / 99
top of WHILE loop

Endpoint Response Code: 200
While loop enpoint called: index 0 / 99
data key found a

In [40]:
#return_stats_of_file(5)
analyze_top_retweeted_tweets_in_data(99)

{'1495556777399955456': 152291,
 '1523510929614270464': 114308,
 '1499360230245015553': 109902,
 '1498962172403384320': 108913,
 '1577718415967621125': 101307,
 '1493848113232367618': 95535,
 '1357241340313141249': 93362,
 '1501938504955154434': 92524,
 '1491605958971191298': 92241,
 '1505925822800007170': 85707,
 '1496456914057715714': 83558,
 '1455174998264586243': 82551,
 '1507273391354814465': 81977,
 '1586892306271326208': 81039,
 '1591016068054908928': 79696,
 '1578436533668888577': 76900,
 '1593933916955705345': 75822,
 '1497549644448301057': 73007,
 '1421416422794747909': 72224,
 '1502980257275875330': 71299,
 '1521838876872278017': 70948,
 '1594674635311824898': 67913,
 '1603699342510014464': 64197,
 '1587010837411115009': 63756,
 '1508981405229129728': 63519,
 '1513831014040477706': 62356,
 '1594707387805417472': 61134,
 '1491067405506068480': 59863,
 '1505927905196478466': 59022,
 '1589958993497456646': 58227,
 '1507311786324033550': 57390,
 '1429320557393178626': 57120,
 '1